In [53]:
import math
import numpy as np
import pandas as pd
validator = "Twinstake_processed.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [54]:
total_missed = 0
indices = df_data['block_num']
for i in range(1,len(indices)):
    if int(indices[i]) != int(indices[i-1])+1:
        total_missed += 1
        print(i)

In [55]:
print(total_missed)
print(len(df_data))

0
100


In [30]:
df_data.reset_index(inplace=True)

In [31]:
print(df_data)

    index  block_num  Block_length_target  Goal_Bonded  Inflation_Rate_Change  \
0       0   12711453             7.233028       0.6666                   0.13   
1       1   12711454             7.233028       0.6666                   0.13   
2       2   12711455             7.233028       0.6666                   0.13   
3       3   12711456             7.233028       0.6666                   0.13   
4       4   12711457             7.233028       0.6666                   0.13   
..    ...        ...                  ...          ...                    ...   
95     95   12711548             7.233028       0.6666                   0.13   
96     96   12711549             7.233028       0.6666                   0.13   
97     97   12711550             7.233028       0.6666                   0.13   
98     98   12711551             7.233028       0.6666                   0.13   
99     99   12711552             7.233028       0.6666                   0.13   

    Min_Inflation_Rate  Max

In [32]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

   block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0      12711453       185752     184.815689                                  
1      12711454       185752     184.815689                                  
2      12711455       185752     184.823787                                  
3      12711456       185752     184.839936                                  
4      12711457       185752     184.839936                                  

  Realized APY Expected APY Absolute Difference in Rewards  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            

  Absolute Difference in APY % Difference in Rewards  
0                                                     
1                                               

In [33]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

    block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0       12711453       185752     184.815689                                  
1       12711454       185752     184.815689           0.0         0.008184   
2       12711455       185752     184.823787      0.008098         0.008167   
3       12711456       185752     184.839936       0.01615         0.008154   
4       12711457       185752     184.839936           0.0         0.008157   
..           ...          ...            ...           ...              ...   
95      12711548       185752     185.575111      0.008083         0.008154   
96      12711549       185752     185.583180      0.008069         0.008154   
97      12711550       185752     185.591249      0.008069         0.008157   
98      12711551       185752     185.607393      0.016145         0.008159   
99      12711552       185752     185.607393           0.0          0.00816   

   Realized APY Expected APY Absolute Difference in

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_8305/160660359.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_8305/160660359.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Expected Rewards'][i] = E_rewards
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_8305/160660359.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [34]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [35]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [36]:
(total_rewards-expected_total)/total_rewards*100

-2.0751042022820845

In [37]:
df_output[1:][df_output['Epoch Rewards'][1:]>0.010]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
3,12711456,185752,184.839936,0.01615,0.008154,24.55302,12.396865,-0.007996,-12.156155,-49.509816
14,12711467,185752,184.928827,0.016165,0.008165,42.1277,21.278555,-0.008,-20.849145,-49.490347
16,12711469,185752,184.944976,0.016148,0.008162,41.438827,20.94392,-0.007987,-20.494908,-49.458223
22,12711475,185752,184.993451,0.016164,0.00816,22.625291,11.421724,-0.008004,-11.203567,-49.517891
29,12711482,185752,185.050023,0.016166,0.008154,43.530842,21.956548,-0.008012,-21.574293,-49.560938
43,12711496,185752,185.163154,0.016168,0.008174,40.797839,20.626762,-0.007994,-20.171077,-49.441533
84,12711537,185752,185.494338,0.016172,0.008167,42.239592,21.331552,-0.008005,-20.90804,-49.498678
98,12711551,185752,185.607393,0.016145,0.008159,42.130054,21.2899,-0.007986,-20.840154,-49.46624


In [40]:
df_output[10:20]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
10,12711463,185752,184.888402,0.008079,0.008154,20.866384,21.061289,0.000075,0.194905,0.93406
11,12711464,185752,184.896494,0.008092,0.008154,20.491527,20.64974,0.000062,0.158213,0.772092
12,12711465,185752,184.904592,0.008098,0.008154,21.428274,21.576663,0.000056,0.148389,0.692491
13,12711466,185752,184.912662,0.008071,0.008172,21.37911,21.647286,0.000101,0.268176,1.254383
14,12711467,185752,184.928827,0.016165,0.008165,42.1277,21.278555,-0.008,-20.849145,-49.490347
15,12711468,185752,184.928827,0.0,0.008154,0.0,21.090173,0.008154,21.090173,inf
16,12711469,185752,184.944976,0.016148,0.008162,41.438827,20.94392,-0.007987,-20.494908,-49.458223
17,12711470,185752,184.953050,0.008075,0.00816,21.289901,21.51536,0.000086,0.225459,1.058995
18,12711471,185752,184.953050,0.0,0.008161,0.0,21.618065,0.008161,21.618065,inf
19,12711472,185752,184.961126,0.008075,0.00817,21.153805,21.402797,0.000095,0.248992,1.177053


In [46]:
df_output['Epoch Rewards'][1:100].mean()

0.007997013852352011

In [47]:
df_output['Expected Rewards'][1:20].mean()

0.008162685291599376